In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

##### Input variables

In [2]:
v1 = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\variables.csv")

##### Construction

In [3]:
v1 = v1.rename(columns={'GVKEY': 'gvkey'})

#1.
v1['accrual_wr'] = (v1['oancfy']-v1['ibq'])/v1['atq']

#2.
v1['aftret_eq_wr'] = v1['ibcomq']/v1['ceqq']

#3.
v1['aftret_equity_wr'] = v1['ibq']/v1['seqq']

#4.
df = v1.sort_values(by=['gvkey', 'datadate'])  # 先按公司和日期排序
# 計算 lag 變數
df['l_icaptq'] = df.groupby('gvkey')['icaptq'].shift(1)  # 上一期的 Invested Capital
df['l_txditcq'] = df.groupby('gvkey')['txditcq'].shift(1)  # 上一期的 Deferred Taxes
df['l_mibq'] = df.groupby('gvkey')['mibq'].shift(1)  # 上一期的 Minority Interest Balance
# 計算分子
df['numerator'] = df['ibq'] + df['xintq'] + df['miiq']
# 計算分母
df['denominator'] = df['l_icaptq'] + df['l_txditcq'] - df['l_mibq']
# 計算 After-tax Return on Invested Capital，分母為正時才計算
df['after_tax_roic'] = df['numerator'] / df['denominator']
df.loc[df['denominator'] <= 0, 'after_tax_roic'] = None  # 分母為負或零時設為 None

#5.
v1['at_turn_wr'] = v1['saleq']/v1['atq']

#6.
v1['capital_ratio_wr'] = v1['dlttq']/(v1['dlttq']+v1['ceqq']+v1['pstkq'])

#7.
# 計算 lag 變數（上一期的值）
df['l_invtq'] = df.groupby('gvkey')['invtq'].shift(1)
df['l_rectq'] = df.groupby('gvkey')['rectq'].shift(1)
df['l_apq'] = df.groupby('gvkey')['apq'].shift(1)

# 計算 Days Inventory Outstanding (DIO)
df['DIO'] = (df['invtq'] + df['l_invtq']) / 2 / (df['cogsq'] / 365)

# 計算 Days Sales Outstanding (DSO)
df['DSO'] = (df['rectq'] + df['l_rectq']) / 2 / (df['saleq'] / 365)

# 計算 Days Payable Outstanding (DPO)
df['DPO'] = (df['apq'] + df['l_apq']) / 2 / (df['cogsq'] / 365)

# 計算 Cash Conversion Cycle (CCC)
df['cash_conversion_wr'] = df['DIO'] + df['DSO'] - df['DPO']

#8.
v1['cash_debt_wr'] = (v1['ibq']+v1['dpq'])/v1['ltq']

#9.
v1['cash_lt_wr'] = v1['cheq']/v1['ltq']

#10.
v1['cash_ratio_wr'] = v1['cheq']/v1['lctq']

#11.
v1['cfm_wr'] = (v1['ibq']+v1['dpq'])/v1['saleq']

#12.
v1['curr_debt_wr'] = v1['lctq']/v1['ltq']

#13.
v1['curr_ratio_wr'] = v1['actq']/v1['lctq']

#14.
v1['de_ratio_wr'] = v1['ltq']/(v1['ceqq']+v1['pstkq'])

#15.
v1['debt_assets_wr'] = v1['ltq']/v1['atq']

#16.
v1['debt_at_wr'] = (v1['dlttq']+v1['dlcq'])/v1['atq']

#17.
v1['debt_book_value'] = v1['dlttq']+v1['dlcq']+v1['pstkq']

#18.
v1['atq_wr'] = v1['atq']

#19.
v1['ceqq_wr'] = v1['ceqq']

#20. 
v1['saleq_wr'] = v1['saleq'] 

#21. 
v1['total_debt_wr'] = v1['dlttq']+v1['dlcq']

In [4]:
# 使用 'gvkey' 和 'datadate' 作為鍵進行合併
v1 = v1.merge(df[['gvkey', 'datadate', 'after_tax_roic','cash_conversion_wr']], on=['gvkey', 'datadate'], how='left')
v1

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,cusip,curcdq,...,de_ratio_wr,debt_assets_wr,debt_at_wr,debt_book_value,atq_wr,ceqq_wr,saleq_wr,total_debt_wr,after_tax_roic,cash_conversion_wr
0,1001,1983-09-30,1983,3,INDL,C,D,STD,000165100,USD,...,3.815494,0.792337,0.621190,5.869,9.448,1.962,6.449,5.869,NaN,NaN
1,1001,1983-12-31,1983,4,INDL,C,D,STD,000165100,USD,...,0.799821,0.444389,0.345455,4.864,14.080,7.823,8.166,4.864,0.072727,-81.922742
2,1001,1984-03-31,1984,1,INDL,C,D,STD,000165100,USD,...,0.749629,0.428450,0.326590,4.617,14.137,8.080,6.434,4.617,0.031972,-78.042938
3,1001,1984-06-30,1984,2,INDL,C,D,STD,000165100,USD,...,0.781540,0.438688,0.311114,4.714,15.152,8.505,7.559,4.714,0.047483,-96.095643
4,1001,1984-09-30,1984,3,INDL,C,D,STD,000165100,USD,...,0.815784,0.449274,0.284451,4.601,16.175,8.908,8.058,4.601,0.043286,-157.804838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001173,332115,2020-03-31,2020,1,INDL,C,D,STD,04216R102,USD,...,0.317185,0.240805,0.057866,2.552,44.102,33.482,0.000,2.552,-0.255865,inf
1001174,335466,2019-06-30,2019,2,INDL,C,D,STD,G4511M108,USD,...,2.749699,0.733312,0.689710,141.905,205.746,54.870,12.713,141.905,NaN,NaN
1001175,335466,2019-09-30,2019,3,INDL,C,D,STD,G4511M108,USD,...,3.157922,0.759495,0.719683,141.905,197.177,47.422,11.750,141.905,-0.062675,224.909888
1001176,335466,2019-12-31,2019,4,INDL,C,D,STD,G4511M108,USD,...,2.873331,0.741824,0.702817,141.905,201.909,52.128,12.790,141.905,-0.002299,153.167682


In [5]:
#1. 
v1['quick_ratio_wr'] = (v1['actq']-v1['invtq'])/v1['lctq']

#2. 
# 替換 xrd 的缺失值為 0
v1['xrdq'] = v1['xrdq'].fillna(0)
# 計算 R&D/Sales 比率並確保最小值為 0
v1['rd_sale_wr'] = (v1['xrdq'] / v1['saleq']).clip(lower=0)

#3.
v1['rect_act_wr'] = v1['rectq']/v1['actq']

#4.
df = v1.sort_values(by=['gvkey', 'datadate'])
df['l_rectq'] = df.groupby('gvkey')['rectq'].shift(1)  
df['avg_rectq'] = (df['rectq'] + df['l_rectq']) / 2
df['rect_turn_wr'] = (df['saleq']) / df['avg_rectq']
df.loc[df['avg_rectq'] == 0, 'rect_turn_wr'] = None  

#5.
v1['roa_wr'] = v1['oibdpq']/v1['atq']

#6.
v1['roce_wr'] = v1['oiadpq']/(v1['dlttq']+v1['pstkq']+v1['dlcq']+v1['ceqq'])

#7.
v1['roe_wr'] = v1['ibq']/(v1['atq']-v1['ltq'])

#8. 
v1['short_debt_wr'] = v1['dlcq']/(v1['dlttq']+v1['dlcq'])

#9. 
v1['totdebt_invcap_wr'] = (v1['dlttq']+v1['dlcq'])/v1['icaptq']
v1.loc[v1['icaptq'] <= 0, 'totdebt_invcap_wr'] = None

In [6]:
v1 = v1.merge(df[['gvkey', 'datadate', 'rect_turn_wr']], on=['gvkey', 'datadate'], how='left')
v1

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,cusip,curcdq,...,cash_conversion_wr,quick_ratio_wr,rd_sale_wr,rect_act_wr,roa_wr,roce_wr,roe_wr,short_debt_wr,totdebt_invcap_wr,rect_turn_wr
0,1001,1983-09-30,1983,3,INDL,C,D,STD,000165100,USD,...,NaN,0.515319,0.0,0.009057,0.085203,0.083131,0.185525,0.247231,0.919906,NaN
1,1001,1983-12-31,1983,4,INDL,C,D,STD,000165100,USD,...,-81.922742,2.343962,0.0,0.017058,0.075639,0.046977,0.031829,0.106908,0.399770,164.969697
2,1001,1984-03-31,1984,1,INDL,C,D,STD,000165100,USD,...,-78.042938,1.980635,0.0,0.030047,0.043078,0.027644,0.031807,0.090752,0.376038,63.389163
3,1001,1984-06-30,1984,2,INDL,C,D,STD,000165100,USD,...,-96.095643,1.595447,0.0,0.011350,0.067318,0.053030,0.049971,0.076156,0.366563,90.526946
4,1001,1984-09-30,1984,3,INDL,C,D,STD,000165100,USD,...,-157.804838,1.309201,0.0,0.009930,0.058918,0.046043,0.045240,0.072810,0.349249,185.241379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1007725,332115,2020-03-31,2020,1,INDL,C,D,STD,04216R102,USD,...,inf,4.054743,inf,0.038796,-0.104893,-0.136565,-0.151664,0.535266,0.073613,0.000000
1007726,335466,2019-06-30,2019,2,INDL,C,D,STD,G4511M108,USD,...,NaN,0.148579,0.0,0.377669,0.008885,-0.004228,-0.072499,1.000000,2.586204,NaN
1007727,335466,2019-09-30,2019,3,INDL,C,D,STD,G4511M108,USD,...,224.909888,0.112975,0.0,0.491147,0.003256,-0.012798,-0.118911,1.000000,2.992387,1.261474
1007728,335466,2019-12-31,2019,4,INDL,C,D,STD,G4511M108,USD,...,153.167682,2.708771,0.0,0.353584,0.005557,-0.009194,-0.043796,0.001459,0.732126,1.456471


In [ ]:
#1.
v1['debt_capital_wr'] = (v1['apq']+v1['dlcq']+v1['dlttq'])/(v1['apq']+v1['dlcq']+v1['dlttq']+v1['ceqq']+v1['pstkq'])

#2.
v1['debt_ebitda_wr'] = (v1['dlttq']+v1['dlcq'])/v1['oibdpq']

#3.
v1['debt_invcap_wr'] = v1['dlttq']/v1['icaptq']

#4. 
v1['dltt_be_wr'] = v1['dlttq']/(v1['atq']-v1['ltq'])

#5. 
v1['dpr_wr'] = v1['dvy']/v1['ibadjq']

#6.
v1['efftax_wr'] = v1['txtq']/v1['piq']

#7.
v1['equity_invcap_wr'] = v1['ceqq']/v1['icaptq']

#8.
v1['fcf_ocf_wr'] = (v1['oancfy']-v1['capxy'])/v1['oancfy']

#9.
v1['gpm_wr'] = (v1['saleq']-v1['cogsq'])/v1['saleq']

#10.
v1['gprof_wr'] = (v1['saleq']-v1['cogsq'])/v1['atq']

#11.
v1['int_debt_wr'] = v1['xintq']/v1['dlttq']

#12.
v1['int_totdebt_wr'] = v1['xintq']/(v1['dlttq']+v1['dlcq'])

#13.
v1['intcov_ratio_wr'] = v1['oiadpq']/v1['xintq']

#14.
v1['intcov_wr'] = (v1['xintq']+v1['ibq'])/v1['xintq']

#15.v1
v1['invt_act_wr'] = v1['invtq']/v1['actq']

#16.
df = v1.sort_values(by=['gvkey', 'datadate'])  # 先按公司和日期排序
# 計算 lag 變數
df['l_invtq'] = df.groupby('gvkey')['invtq'].shift(1) 
# 計算分子
df['numerator'] = df['cogsq'] 
# 計算分母
df['denominator'] = (df['invtq'] + df['l_invtq'])/2
# 分母為正時才計算
df['invturn_wr'] = df['numerator'] / df['denominator']
df.loc[df['denominator'] <= 0, 'invturn_wr'] = None  # 分母為負或零時設為 None

#17.
v1['lt_debt_wr'] = v1['dlttq']/v1['ltq']

#18.
v1['lt_ppent_wr'] = v1['ltq']/v1['ppentq']

#19.
v1['npm_wr'] = v1['ibq']/v1['saleq']

#20.
v1['ocf_lct_wr'] = v1['oancfy']/v1['lctq']

#21.
v1['opmad_wr'] = v1['oiadpq']/v1['saleq']

#22.
v1['opmbd_wr'] = v1['oibdpq']/v1['saleq']

#23.
df = df.sort_values(by=['gvkey', 'datadate'])

# 計算存貨的變動值 (D_INVT)
df['d_invtq'] = df['invtq'] - df['l_invtq']  # 當期存貨變動
# 計算上期應付賬款 (L_AP)
df['l_apq'] = df.groupby('gvkey')['apq'].shift(1)  # 上一期的應付賬款
# 計算應付賬款的平均值
df['avg_apq'] = (df['apq'] + df['l_apq']) / 2
# 計算 Payables Turnover，當 avg_ap > 0 才計算
df['pay_turn_wr'] = (df['cogsq'] + df['d_invtq']) / df['avg_apq']
df.loc[df['avg_apq'] <= 0, 'pay_turn_wr'] = None  # 當 avg_ap 小於或等於 0 時設為 None

#24.
v1['pretret_earnat_wr'] = v1['oibdpq']/(v1['ppentq']+v1['actq'])

#25.
v1['pretret_noa_wr'] = v1['oiadpq']/(v1['ppentq']+v1['actq']-v1['lctq'])

#26.
v1['profit_lct_wr'] = v1['oibdpq']/v1['lctq']

#27.
v1['ptpm_wr'] = (v1['oiadpq']-v1['xintq']+v1['spiq']+v1['nopiq'])/v1['saleq']

#28.
v1['mkvaltq'] = v1['prccq']*v1['cshoq']
v1['m2b'] = v1['mkvaltq']/v1['ceqq']
v1['v2a'] = (v1['mkvaltq']+v1['dlttq']+v1['dlcq']-v1['cheq'])/v1['atq']
v1['v2s'] = (v1['mkvaltq']+v1['dlttq']+v1['dlcq'])/v1['saleq']
# v1 = v1[(v1['m2b'] >= 0) | (v1['v2a'] >= 0) | (v1['v2s'] >= 0)]
v1['lnm2b_y'] = np.log10(v1['m2b'])
v1['lnv2a_y'] = np.log10(v1['v2a'])
v1['lnv2s_y'] = np.log10(v1['v2s'])

c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
v1 = v1.merge(df[['gvkey', 'datadate', 'invturn_wr','pay_turn_wr']], on=['gvkey', 'datadate'], how='left')
v1

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,cusip,curcdq,...,ptpm_wr,mkvaltq,m2b,v2a,v2s,lnm2b_y,lnv2a_y,lnv2s_y,invturn_wr,pay_turn_wr
0,1001,1983-09-30,1983,3,INDL,C,D,STD,000165100,USD,...,0.087300,15.667750,7.985601,2.130266,3.339549,0.902308,0.328434,0.523688,NaN,NaN
1,1001,1983-12-31,1983,4,INDL,C,D,STD,000165100,USD,...,0.042861,25.868000,3.306660,1.878693,3.763409,0.519390,0.273856,0.575581,6.174475,2.584000
2,1001,1984-03-31,1984,1,INDL,C,D,STD,000165100,USD,...,0.061859,22.746000,2.815099,1.700856,4.252875,0.449494,0.230668,0.628683,4.694611,2.291066
3,1001,1984-06-30,1984,2,INDL,C,D,STD,000165100,USD,...,0.091414,20.516000,2.412228,1.438490,3.337743,0.382418,0.157907,0.523453,4.989390,2.178052
4,1001,1984-09-30,1984,3,INDL,C,D,STD,000165100,USD,...,0.070861,16.502000,1.852492,1.093849,2.618888,0.267756,0.038957,0.418117,4.798155,1.584077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143805,332115,2020-03-31,2020,1,INDL,C,D,STD,04216R102,USD,...,-inf,57.799500,1.726286,0.819521,inf,0.237113,-0.086440,inf,NaN,4.639920
1143806,335466,2019-06-30,2019,2,INDL,C,D,STD,G4511M108,USD,...,-0.312908,55.445500,1.010488,0.906295,15.523519,0.004531,-0.042731,1.190990,NaN,NaN
1143807,335466,2019-09-30,2019,3,INDL,C,D,STD,G4511M108,USD,...,-0.479915,25.895700,0.546069,0.814399,14.280911,-0.262752,-0.089163,1.154756,4.574739,2.322735
1143808,335466,2019-12-31,2019,4,INDL,C,D,STD,G4511M108,USD,...,-0.178499,26.105953,0.500805,0.769307,13.136118,-0.300332,-0.113900,1.118467,5.299583,2.199516


##### summary statistics table

In [9]:
# pd.set_option('display.max_rows', None)
# Ensure 'datadate' is in datetime format
v1['datadate'] = pd.to_datetime(v1['datadate'], errors='coerce')

# Extract month-year period
v1['month'] = v1['datadate'].dt.to_period('M')

# Calculate missing and non-missing value counts for each column grouped by month-year
missing_values_count = v1.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.isnull().sum())
non_missing_values_count = v1.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.notnull().sum())

# Display both counts side by side
combined_counts = pd.concat([missing_values_count, non_missing_values_count], axis=1, keys=['Missing', 'Not Missing'])

# Print the combined DataFrame
combined_counts

Missing                                                   Not Missing  \
        mkvaltq ceqq dlttq  dlcq cheq  atq saleq  m2b   v2a   v2s     mkvaltq   
month                                                                           
1980-06      96  103    86   280  102   55    24  138   340   327        1905   
1980-07      15   13    11    49   12    7     4   19    60    59         332   
1980-08      12   10    10    33   13    8     3   14    40    39         220   
1980-09     484  493   474   672  491  441    31  526   734   723        2529   
1980-10      18   15    15    54   15    9     5   21    63    63         329   
...         ...  ...   ...   ...  ...  ...   ...  ...   ...   ...         ...   
2020-01     102  106   107   109  106  106   106  107   110   110         249   
2020-02      75   82    82    84   82   82    82   84    86    86         119   
2020-03     739  779   816  1217  764  760   492  793  1252  1258        4713   
2020-04     102  105   106   110  105  105   105  106   112   112         246   
2020-05      77   82    82    85   82   82    82   85    88    88         115   

                                                               
         ceqq dlttq  dlcq  cheq   atq saleq   m2b   v2a   v2s  
month                                                          
1980-06  1898  1915  1721  1899  1946  1977  1863  1661  1674  
1980-07   334   336   298   335   340   343   328   287   288  
1980-08   222   222   199   219   224   229   218   192   193  
1980-09  2520  2539  2341  2522  2572  2982  2487  2279  2290  
1980-10   332   332   293   332   338   342   326   284   284  
...       ...   ...   ...   ...   ...   ...   ...   ...   ...  
2020-01   245   244   242   245   245   245   244   241   241  
2020-02   112   112   110   112   112   112   110   108   108  
2020-03  4673  4636  4235  4688  4692  4960  4659  4200  4194  
2020-04   243   242   238   243   243   243   242   236   236  
2020-05   110   110   107   110   110   110   107   104   104  

[480 rows x 20 columns]

In [10]:
# Calculate total missing and non-missing counts for each column across the entire DataFrame
missing_values_count = v1[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].isnull().sum()
non_missing_values_count = v1[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].notnull().sum()

# Calculate missing ratio for each column
missing_ratio = round(missing_values_count / (missing_values_count + non_missing_values_count),2)

# Combine results into a DataFrame
missing_ratio_df = pd.DataFrame({
    'Missing Count': missing_values_count,
    'Not Missing Count': non_missing_values_count,
    'Missing Ratio': missing_ratio
})

# Display the DataFrame
missing_ratio_df

,Missing Count,Not Missing Count,Missing Ratio
mkvaltq,79724,1064086,0.07
ceqq,124886,1018924,0.11
dlttq,129865,1013945,0.11
dlcq,186658,957152,0.16
cheq,126276,1017534,0.11
atq,119564,1024246,0.10
saleq,107777,1036033,0.09
m2b,145049,998761,0.13
v2a,210553,933257,0.18
v2s,212199,931611,0.19


##### 篩選欄位

In [11]:
# 篩選出以 'wr' 結尾的欄位
wr_columns = [col for col in v1.columns if col.endswith('wr')]
target_columns = ['lnm2b_y', 'lnv2a_y', 'lnv2s_y']

# 指定要保留的其他欄位
other_columns = ['gvkey', 'datadate', 'fyearq', 'fqtr', 'sic']

# 合併兩個欄位列表
columns_to_keep = other_columns + wr_columns + target_columns

# 篩選 DataFrame 中指定的欄位
df = v1[columns_to_keep]
df.head()

,gvkey,datadate,fyearq,fqtr,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,at_turn_wr,capital_ratio_wr,...,opmbd_wr,pretret_earnat_wr,pretret_noa_wr,profit_lct_wr,ptpm_wr,invturn_wr,pay_turn_wr,lnm2b_y,lnv2a_y,lnv2s_y
0,1001,1983-09-30,1983,3,5812,NaN,0.185525,0.185525,0.682578,0.692476,...,0.124826,0.095583,0.121591,0.262386,0.087300,NaN,NaN,0.902308,0.328434,0.523688
1,1001,1983-12-31,1983,4,5812,NaN,0.031829,0.031829,0.579972,0.357031,...,0.130419,0.079817,0.052143,0.556717,0.042861,6.174475,2.584000,0.519390,0.273856,0.575581
2,1001,1984-03-31,1984,1,5812,NaN,0.031807,0.031807,0.455118,0.341912,...,0.094653,0.045492,0.030448,0.327595,0.061859,4.694611,2.291066,0.449494,0.230668,0.628683
3,1001,1984-06-30,1984,2,5812,NaN,0.049971,0.049971,0.498878,0.338647,...,0.134938,0.070873,0.057896,0.446585,0.091414,4.989390,2.178052,0.382418,0.157907,0.523453
4,1001,1984-09-30,1984,3,5812,NaN,0.045240,0.045240,0.498176,0.323820,...,0.118268,0.061787,0.049287,0.339872,0.070861,4.798155,1.584077,0.267756,0.038957,0.418117


In [12]:
# 先將無窮大值轉換為 NaN
v1.replace([np.inf, -np.inf], np.nan, inplace=True)

# 過濾掉 NaN 比例超過 90% 的欄位
# valid_columns = v1.columns[v1.isnull().mean() < 0.9]
numeric_cols = wr_columns + target_columns

# 計算 summary statistics，忽略 NaN
summary_stats = v1[numeric_cols].agg([
    'count', 
    lambda x: x.mean(skipna=True), 
    lambda x: x.quantile(0.01), 
    lambda x: x.quantile(0.10), 
    lambda x: x.median(skipna=True), 
    lambda x: x.quantile(0.90), 
    lambda x: x.quantile(0.99)
])

# 重命名 index
summary_stats.index = ['N', 'Mean', 'p1', 'p10', 'p50', 'p90', 'p99']

# 設置顯示格式
summary_stats.loc['N'] = summary_stats.loc['N'].astype(int)  # 將 'N' 設為整數
pd.options.display.float_format = '{:.2f}'.format  # 其他值顯示到小數點後兩位

summary_stats

,accrual_wr,aftret_eq_wr,aftret_equity_wr,at_turn_wr,capital_ratio_wr,cash_debt_wr,cash_lt_wr,cash_ratio_wr,cfm_wr,curr_debt_wr,...,opmbd_wr,pretret_earnat_wr,pretret_noa_wr,profit_lct_wr,ptpm_wr,invturn_wr,pay_turn_wr,lnm2b_y,lnv2a_y,lnv2s_y
N,728665.00,1015020.00,1020139.00,1017682.00,1008542.00,859122.00,1016629.00,846150.00,844940.00,849245.00,...,863205.00,729169.00,826927.00,736737.00,738585.00,704504.00,916457.00,960636.00,915232.00,905097.00
Mean,0.01,0.58,0.59,0.25,0.30,0.03,1.29,2.07,-3.64,0.59,...,-3.46,0.01,18958179262.37,0.10,-4.01,8.90,4.40,0.29,-0.01,0.86
p1,-0.55,-1.58,-1.54,0.00,0.00,-2.00,0.00,0.00,-24.43,0.07,...,-21.79,-0.60,-1.40,-2.56,-25.23,0.03,-0.00,-0.58,-1.29,-0.24
p10,-0.09,-0.15,-0.15,0.02,0.00,-0.20,0.01,0.03,-0.46,0.19,...,-0.33,-0.09,-0.15,-0.38,-0.62,0.38,0.04,-0.15,-0.52,0.22
p50,0.02,0.02,0.02,0.20,0.22,0.03,0.13,0.39,0.08,0.58,...,0.12,0.03,0.03,0.11,0.04,1.29,1.95,0.25,-0.01,0.80
p90,0.12,0.08,0.07,0.51,0.67,0.16,2.25,4.05,0.28,0.99,...,0.41,0.09,0.12,0.42,0.25,10.02,5.75,0.80,0.50,1.51
p99,0.42,1.07,0.98,1.13,1.34,0.80,16.77,24.08,0.94,1.00,...,0.85,0.21,1.21,1.35,0.94,94.16,23.77,1.58,1.11,2.87


In [13]:
# 轉置 summary_stats 並匯出 CSV
summary_stats_transposed = summary_stats.T  # 轉置 DataFrame
# summary_stats_transposed.to_csv('summary_statistics.csv', index=True)  # 匯出 CSV 檔案，保留索引

-----

##### Target variables

In [ ]:
# Ensure 'datadate' is in datetime format
df['datadate'] = pd.to_datetime(df['datadate'], errors='coerce')

# Extract month-year period
df['month'] = df['datadate'].dt.to_period('M')

# Calculate missing and non-missing value counts for each column grouped by month-year
missing_values_count = df.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.isnull().sum())
non_missing_values_count = df.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.notnull().sum())

# Display both counts side by side
combined_counts = pd.concat([missing_values_count, non_missing_values_count], axis=1, keys=['Missing', 'Not Missing'])

# Print the combined DataFrame
combined_counts

In [ ]:
selected_t1 = t1[['m2b','v2a','v2s','lnm2b','lnv2a','lnv2s','gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'curcdq', 'datacqtr', 'datafqtr','costat', 'naics', 'sic']]
df = df.merge(selected_t1, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic'], how='left')
df

##### 各步驟篩選後的樣本量

In [ ]:
# 加上cusip
cusip = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\cusip.csv")
cusip = cusip[['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic','cusip']]
# cusip['month'] = pd.to_datetime(cusip['datadate']).dt.to_period('M')
df = df.merge(cusip, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic'], how='left')
df

In [ ]:
# drop cusip if it is na
df = df.dropna(subset=['cusip'])

df1 = df[['gvkey','cusip']]
df1 = df1.drop_duplicates()
# 根據 cusip 欄位進行排序
df1 = df1.sort_values(by='cusip')
df1 = df1.reset_index(drop=True)
df2 = df1['cusip']
# 將 DataFrame 中的單一欄位保存到純文字文件
# df2.to_csv("output.txt", index=False)

In [ ]:
df1, df2

In [ ]:
cusip = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\cusip convert.csv")
cusip = cusip.rename(columns={'cusip': 'crsp_cusip'})
cusip

In [ ]:
df1 = df1.rename(columns={'cusip': 'compstat_cusip'})
merged_df = pd.merge(df1, cusip, left_index=True, right_index=True, how='inner')
new_df = df.merge(merged_df, on=['gvkey'], how='left')
new_df
# new_df is updated cusip df

##### 把new_df存成csv，以後執行以下程式碼即可

In [ ]:
# new_df.to_csv('new_df.csv', index=False)
new_df = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\new_df.csv")

In [ ]:
new_df

In [ ]:
s_e = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\share&exc code.csv")
s_e = s_e[['CUSIP','SHRCD','EXCHCD']]
# 使用 drop_duplicates 刪除重複項
s_e = s_e.drop_duplicates()

In [ ]:
model_df = pd.merge(new_df, s_e, left_on='crsp_cusip', right_on='CUSIP', how='left')
print(model_df)

In [ ]:
# 1. 第一步：所有美國普通股上市公司（在指定交易所）
step1_df = model_df[(model_df['EXCHCD'].isin([1, 2, 3])) & (model_df['SHRCD'].isin([10, 11]))]
step1_firm_months = len(step1_df)
step1_unique_firms = step1_df['gvkey'].nunique()

# 2. 第二步：目標變數（m2b, v2a, v2s）為正且不缺失
step2_df = step1_df.dropna(subset=['lnm2b', 'lnv2a', 'lnv2s'])
# step2_df = step2_df[(step2_df['lnm2b'] > 0) & (step2_df['lnv2a'] > 0) & (step2_df['lnv2s'] > 0)]
step2_firm_months = len(step2_df)
step2_unique_firms = step2_df['gvkey'].nunique()

# 3. 第三步：每月所有規模變數位於第 10 百分位以上
# 提取每月的第 10 百分位值並進行篩選
step2_df['month'] = pd.to_datetime(step2_df['datadate']).dt.to_period('M')
percentiles = step2_df.groupby('month')[['atq_wr', 'saleq_wr', 'ceqq_wr']].quantile(0.1).rename(columns={
    'atq_wr': 'atq_wr_10th', 'saleq_wr': 'saleq_wr_10th', 'ceqq_wr': 'ceqq_wr_10th'
}).reset_index()

# 合併百分位數並篩選符合條件的觀察值
step3_df = step2_df.merge(percentiles, on='month', how='left')
step3_df = step3_df[(step3_df['atq_wr'] >= step3_df['atq_wr_10th']) & 
                    (step3_df['saleq_wr'] >= step3_df['saleq_wr_10th']) & 
                    (step3_df['ceqq_wr'] >= step3_df['ceqq_wr_10th'])]
step3_firm_months = len(step3_df)
step3_unique_firms = step3_df['gvkey'].nunique()

# 創建表格
summary_table = pd.DataFrame({
    '步驟': [
        '在紐約證券交易所（NYSE）、納斯達克（NASDAQ）或美國證券交易所（AMEX）上市的美國普通股',
        '目標變數（m2b, v2a, v2s）為正且不缺失',
        '每月所有規模變數（資產、賬面權益或銷售額）位於第 10 百分位以上'
    ],
    '公司-月份觀察值數量': [step1_firm_months, step2_firm_months, step3_firm_months],
    '公司數量': [step1_unique_firms, step2_unique_firms, step3_unique_firms]
})

# 顯示結果
print(summary_table)

In [ ]:
step3_df.columns

##### add in fama-french 49 industries categories

In [ ]:
sic_data = [
    ["Agric", "0100-0799"],
    ["Agric", "2048-2048"],
    ["Food", "2000-2046"],
    ["Food", "2050-2063"],
    ["Food", "2070-2079"],
    ["Food", "2090-2095"],
    ["Food", "2098-2099"],
    ["Soda", "2064-2068"],
    ["Soda", "2086-2087"],
    ["Soda", "2096-2097"],
    ["Beer", "2080-2085"],
    ["Smoke", "2100-2199"],
    ["Toys", "0900-0999"],
    ["Toys", "3650-3652"],
    ["Toys", "3732-3732"],
    ["Toys", "3930-3949"],
    ["Fun", "7800-7841"],
    ["Fun", "7900-7999"],
    ["Books", "2700-2749"],
    ["Books", "2770-2799"],
    ["Hshld", "2047-2047"],
    ["Hshld", "2391-2392"],
    ["Hshld", "2510-2519"],
    ["Hshld", "2590-2599"],
    ["Hshld", "2840-2844"],
    ["Hshld", "3160-3199"],
    ["Hshld", "3229-3231"],
    ["Hshld", "3260-3260"],
    ["Hshld", "3262-3263"],
    ["Hshld", "3269-3269"],
    ["Hshld", "3630-3639"],
    ["Hshld", "3750-3751"],
    ["Hshld", "3800-3800"],
    ["Hshld", "3860-3879"],
    ["Hshld", "3910-3919"],
    ["Hshld", "3960-3961"],
    ["Hshld", "3991-3991"],
    ["Hshld", "3995-3995"],
    ["Clths", "2300-2390"],
    ["Clths", "3020-3021"],
    ["Clths", "3100-3111"],
    ["Clths", "3130-3159"],
    ["Clths", "3965-3965"],
    ["Hlth", "8000-8099"],
    ["MedEq", "3693-3693"],
    ["MedEq", "3840-3851"],
    ["Drugs", "2830-2836"],
    ["Chems", "2800-2829"],
    ["Chems", "2850-2899"],
    ["Rubbr", "3000-3000"],
    ["Rubbr", "3050-3099"],
    ["Txtls", "2200-2295"],
    ["Txtls", "2297-2299"],
    ["Txtls", "2393-2395"],
    ["Txtls", "2397-2399"],
    ["BIdMt", "0800-0899"],
    ["BIdMt", "2400-2439"],
    ["BIdMt", "2450-2459"],
    ["BIdMt", "2490-2499"],
    ["BIdMt", "2950-2952"],
    ["BIdMt", "3200-3219"],
    ["BIdMt", "3240-3259"],
    ["BIdMt", "3261-3261"],
    ["BIdMt", "3264-3264"],
    ["BIdMt", "3270-3299"],
    ["BIdMt", "3420-3442"],
    ["BIdMt", "3446-3452"],
    ["BIdMt", "3490-3499"],
    ["BIdMt", "3996-3996"],
    ["Construction", "1500-1549"],
    ["Construction", "1600-1699"],
    ["Construction", "1700-1799"],
    ["Steel", "3300-3369"],
    ["Steel", "3390-3399"],
    ["FabPr", "3400-3400"],
    ["FabPr", "3443-3444"],
    ["FabPr", "3460-3479"],
    ["Mach", "3510-3536"],
    ["Mach", "3540-3569"],
    ["Mach", "3580-3599"],
    ["ElcEq", "3600-3621"],
    ["ElcEq", "3623-3629"],
    ["ElcEq", "3640-3646"],
    ["ElcEq", "3648-3649"],
    ["ElcEq", "3660-3660"],
    ["ElcEq", "3691-3692"],
    ["ElcEq", "3699-3699"],
    ["Misc", "3900-3900"],
    ["Misc", "3990-3990"],
    ["Misc", "3999-3999"],
    ["Misc", "9900-9999"],
    ["Autos", "2296-2296"],
    ["Autos", "2396-2396"],
    ["Autos", "3010-3011"],
    ["Autos", "3537-3537"],
    ["Autos", "3647-3647"],
    ["Autos", "3694-3694"],
    ["Autos", "3700-3716"],
    ["Autos", "3790-3792"],
    ["Autos", "3799-3799"],
    ["Aero", "3720-3729"],
    ["Ships", "3730-3731"],
    ["Ships", "3740-3743"],
    ["Guns", "3480-3489"],
    ["Guns", "3760-3769"],
    ["Guns", "3795-3795"],
    ["Gold", "1040-1049"],
    ["Mines", "1000-1039"],
    ["Mines", "1060-1099"],
    ["Mines", "1400-1499"],
    ["Coal", "1200-1299"],
    ["Enrgy", "1310-1389"],
    ["Enrgy", "2900-2911"],
    ["Enrgy", "2990-2999"],
    ["Uti", "4900-4999"],
    ["TeIcm", "4800-4899"],
    ["PerSv", "7020-7021"],
    ["PerSv", "7030-7039"],
    ["PerSv", "7200-7212"],
    ["PerSv", "7215-7299"],
    ["PerSv", "7395-7395"],
    ["PerSv", "7500-7500"],
    ["PerSv", "7520-7549"],
    ["PerSv", "7600-7699"],
    ["PerSv", "8100-8199"],
    ["PerSv", "8200-8299"],
    ["PerSv", "8300-8399"],
    ["PerSv", "8400-8499"],
    ["PerSv", "8600-8699"],
    ["PerSv", "8800-8899"],
    ["BusSv", "2750-2759"],
    ["BusSv", "3993-3993"],
    ["BusSv", "7300-7372"],
    ["BusSv", "7374-7394"],
    ["BusSv", "7397-7397"],
    ["BusSv", "7399-7399"],
    ["BusSv", "7510-7519"],
    ["BusSv", "8700-8748"],
    ["BusSv", "8900-8999"],
    ["Comps", "3570-3579"],
    ["Comps", "3680-3689"],
    ["Comps", "3695-3695"],
    ["Comps", "7373-7373"],
    ["Chips", "3622-3622"],
    ["Chips", "3661-3679"],
    ["Chips", "3810-3810"],
    ["Chips", "3812-3812"],
    ["LabEq", "3811-3811"],
    ["LabEq", "3820-3830"],
    ["Paper", "2520-2549"],
    ["Paper", "2600-2639"],
    ["Paper", "2670-2699"],
    ["Paper", "2760-2761"],
    ["Paper", "3950-3955"],
    ["Boxes", "2440-2449"],
    ["Boxes", "2640-2659"],
    ["Boxes", "3210-3221"],
    ["Boxes", "3410-3412"],
    ["Trans", "4000-4099"],
    ["Trans", "4100-4199"],
    ["Trans", "4200-4299"],
    ["Trans", "4400-4499"],
    ["Trans", "4500-4599"],
    ["Trans", "4600-4699"],
    ["Trans", "4700-4799"],
    ["Whlsl", "5000-5099"],
    ["Whlsl", "5100-5199"],
    ["Rtail", "5200-5299"],
    ["Rtail", "5300-5399"],
    ["Rtail", "5400-5499"],
    ["Rtail", "5500-5599"],
    ["Rtail", "5600-5699"],
    ["Rtail", "5700-5736"],
    ["Rtail", "5900-5999"],
    ["Meals", "5800-5813"],
    ["Meals", "5890-5890"],
    ["Meals", "7000-7019"],
    ["Meals", "7040-7049"],
    ["Meals", "7213-7213"],
    ["Banks", "6000-6099"],
    ["Banks", "6100-6199"],
    ["Insur", "6300-6399"],
    ["Insur", "6400-6411"],
    ["RlBst", "6500-6553"],
    ["Fin", "6200-6299"],
    ["Fin", "6700-6799"]
]
# Create a DataFrame from the data
sic_df = pd.DataFrame(sic_data, columns=["Division", "Range of SIC Codes"])

# Split the 'Range of SIC Codes' into two new columns 'Lower Bound' and 'Upper Bound'
sic_df[['Lower Bound', 'Upper Bound']] = sic_df['Range of SIC Codes'].str.split('-', expand=True)

# Display the resulting DataFrame
print(sic_df[['Lower Bound', 'Upper Bound', 'Division']])

In [ ]:
sic_df['Lower Bound'] = pd.to_numeric(sic_df['Lower Bound'])
sic_df['Upper Bound'] = pd.to_numeric(sic_df['Upper Bound'])

# Function to lookup the division
def find_division(sic_code):
    division = sic_df[(sic_df['Lower Bound'] <= sic_code) & (sic_df['Upper Bound'] >= sic_code)]['Division']
    return division.iloc[0] if not division.empty else 'Unknown'

# Apply the function to the 'sic' column
# 使用 LabelEncoder 編碼產業分類
label_encoder = LabelEncoder()
step3_df['industry'] = step3_df['sic'].apply(find_division)
step3_df['famafrench_wr'] = label_encoder.fit_transform(step3_df['industry'])
step3_df

##### 建立模型

In [ ]:
import numpy as np
import lightgbm as lgb
from numpy import set_printoptions
from sklearn.datasets import make_friedman1
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# 設定了 NumPy 的印出格式，precision=4 代表小數點後保留 4 位數
# linewidth=100 表示每行顯示的最大寬度為 100
set_printoptions(precision=4)
np.set_printoptions(linewidth=100)

In [ ]:
# test settings - 觀測數量、學習率、樹的數量以及每棵樹的葉節點數量
LEARNING_RATE = 0.1
TREES = 12
LEAVES = 5

In [ ]:
# 函數 tree_prediction 用來從梯度提升模型（GBM）中提取單棵樹的預測值
# 根據不同的樹迭代次數，它會返回該樹的預測值
# 如果是第一棵樹（tree_iteration == 0），它返回訓練樣本的平均值；
# 如果是第二棵樹（tree_iteration == 1），它會調整預測值；
# 否則，它會返回指定樹的預測結果
def tree_prediction(model, data, train_sample_average, tree_iteration, learning_rate):
    '''
    extracts individual tree predictions from GBM model
    '''
    if tree_iteration == 0:
        tree_prediction = train_sample_average
    elif tree_iteration == 1:
        gbm1_prediction = model.predict(data, start_iteration=0, num_iteration=1)    
        tree_prediction = (gbm1_prediction - train_sample_average) / learning_rate
    else:
        tree_prediction = model.predict(data, start_iteration=tree_iteration-1, num_iteration=1) / learning_rate
    return tree_prediction

In [ ]:
# 葉節點成員矩陣生成函數
def getD(tree):
    '''
    utility function to create leaf membership matrix D from leaf membership vector tree
    element d_f,c in D takes value of 1 IFF observations f and c are allocated to the same leaf
    that is, tree[f] == tree[c], otherwise 0
    '''
    # 使用 NumPy 來替代雙重迴圈
    tree = np.array(tree)  # 確保 tree 是一個 NumPy 陣列
    D = (tree[:, None] == tree[None, :]).astype(int)
    return D
# getD 會從樹的葉節點生成一個成員矩陣 D。D[f, c] 
# 只有在觀測值 f 和 c 分配到同一個葉節點時才為 1，否則為 0
# tree[:, None] 和 tree[None, :]： 這種方式創建了樹向量的兩個不同視圖，一個是列向量，另一個是行向量。這樣可以進行廣播操作，快速比較每個元素，生成布林矩陣。
# .astype(int)： 將布林矩陣轉換為整數矩陣，以符合原始函數中的邏輯。

In [ ]:
# 選擇特徵欄位和目標欄位
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_column = ['lnm2b']
# target_column = ['lnm2b','lnv2a','lnv2s']
# 生成特徵矩陣 X 和目標變數 y
X = step3_df[feature_columns].to_numpy()
y = step3_df[target_column].to_numpy()

# 確認資料形狀
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 2,
    "learning_rate": LEARNING_RATE,    
}
# boosting_type: 設定提升樹的類型，這裡使用的是 gbdt（Gradient Boosting Decision Tree，梯度提升決策樹）。
# objective: 模型的目標函數，這裡設定為 regression，表示進行回歸分析。
# metric: 評估指標，這裡使用 rmse（均方根誤差）來評估模型的預測誤差。
# num_leaves: 決策樹中葉節點的數量，這裡設定為 LEAVES（之前設定為 5），葉節點越多，模型越複雜。
# verbose: 設定輸出的詳細程度，1 表示顯示基本的運行資訊。
# min_data: 每個葉節點的最小資料量，這裡設定為 2，避免過度擬合。
# learning_rate: 學習率，這裡使用 LEARNING_RATE（之前設定為 0.1），學習率決定了模型每次更新的步伐大小。

In [ ]:
# 訓練 LightGBM 模型
data = lgb.Dataset(X, label=y)
model = lgb.train(params, data, num_boost_round=TREES-1)
y_hat = model.predict(X)
y_hat.shape  # 輸出預測結果的形狀

# 獲取每個樣本在每棵樹中的葉節點編號
instance_leaf_membership = model.predict(X, pred_leaf=True)

In [ ]:
# 生成並顯示每棵樹的結構圖
for index in range(1, TREES):
    graph = lgb.create_tree_digraph(booster=model, tree_index=index-1,
                    show_info=['internal_count', 'leaf_count', 
                               'leaf_weight', 'data_percentage', 'internal_value'],
                    name="Tree"+str(index))
    graph.render(view=True)

In [ ]:
# 提取 GBM 模型中的樹預測
OBSERVATIONS = X.shape[0]  # 將 OBSERVATIONS 設定為資料集的行數
train_average = y.mean()
tree = []
tp = np.full(OBSERVATIONS, train_average)
tree.append(tp)
for index in range(1, TREES):
    tp = tree_prediction(model=model, data=X, train_sample_average=train_average, tree_iteration=index, learning_rate=LEARNING_RATE)
    tree.append(tp)
# train_average：目標變數 y 的平均值，用於初始化第一棵樹的預測值。
# tree_prediction：提取指定樹的預測值，並將結果添加到 tree 列表中。

In [ ]:
ensemble = tree[0]
for index in range(1, TREES):
    ensemble += LEARNING_RATE * tree[index]

# 確認手動構建的預測值與模型的預測值一致
diff = ensemble - y_hat
assert(np.allclose(diff, np.zeros((1, OBSERVATIONS)), atol=1e-6))
# ensemble：用來累加每棵樹的預測值，最後得到整體預測。
# np.allclose：用來檢查兩個數組是否幾乎相等，確認手動預測與模型預測一致。

##### chatgpt

In [ ]:
# 設定樹的數量範圍
tree_options = [10, 20, 50, 100]
best_tree_count = 0
best_rmse = float("inf")

# 定義參數
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "learning_rate": 0.1,
}

# 提取特徵和目標變數
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
X = step3_df[feature_columns].to_numpy()
y = step3_df['lnm2b'].to_numpy()  # 假設用一個目標變數進行測試

# 分割訓練集和驗證集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用不同的樹數進行訓練並記錄 RMSE
for num_trees in tree_options:
    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(params, train_data, num_boost_round=num_trees)
    
    # 驗證集預測
    y_val_pred = model.predict(X_val)
    rmse = np.sqrt(np.mean((y_val_pred - y_val) ** 2))

    print(f"Number of Trees: {num_trees}, RMSE: {rmse}")

    # 記錄最佳樹數
    if rmse < best_rmse:
        best_rmse = rmse
        best_tree_count = num_trees

print(f"Optimal Number of Trees: {best_tree_count}, Best RMSE: {best_rmse}")

In [ ]:
LEARNING_RATE = 0.1
LEAVES = 31
TREES = 100  # 固定迭代次數，設定為100
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 20,
    "learning_rate": LEARNING_RATE,
}

# 假設 step3_df 是你的完整資料集
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_columns = ['lnm2b']
# 添加月份列
step3_df['month'] = pd.to_datetime(step3_df['datadate']).dt.to_period('M')

# 儲存樣本外預測結果
oos_predictions = {target: [] for target in target_columns}

# 每月處理數據
for month, monthly_data in step3_df.groupby('month'):
    print(f"Processing month: {month}")

    # 提取特徵矩陣 X 和多目標變數 y
    X = monthly_data[feature_columns].to_numpy()
    y_all = monthly_data[target_columns].to_numpy()

    # 將資料分為 60% 訓練集、20% 驗證集、20% 測試集
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_all, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # 隨機分配到 5 個區塊
    blocks = np.array_split(np.random.permutation(len(X_train)), 5)

    # 對於每個目標變數，進行五折交叉驗證
    for target_idx, target in enumerate(target_columns):
        print(f"Training models for target: {target}")

        # 訓練資料集
        train_data = lgb.Dataset(X_train, label=y_train[:, target_idx])

        # 使用固定迭代次數訓練模型
        model = lgb.train(
            params,
            train_data,
            num_boost_round=TREES
        )

        # 使用固定迭代次數進行預測
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        # 儲存測試結果
        oos_predictions[target].append({
            'month': month,
            'y_val_pred': y_val_pred,
            'y_test_pred': y_test_pred,
            'y_test_actual': y_test[:, target_idx]
        })

# 最終的樣本外預測結果儲存到 DataFrame 中便於分析
oos_df = pd.DataFrame({
    'month': [entry['month'] for target in target_columns for entry in oos_predictions[target]],
    'target': [target for target in target_columns for _ in oos_predictions[target]],
    'y_val_pred': [entry['y_val_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_pred': [entry['y_test_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_actual': [entry['y_test_actual'] for target in target_columns for entry in oos_predictions[target]]
})

# 查看結果
print(oos_df.head())

##### lnv2s

In [ ]:
LEARNING_RATE = 0.05
LEAVES = 31
TREES = 100  # 固定迭代次數，設定為100
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 20,
    "learning_rate": LEARNING_RATE,
}

# 假設 step3_df 是你的完整資料集
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_columns = ['lnv2s']
# 添加月份列
step3_df['month'] = pd.to_datetime(step3_df['datadate']).dt.to_period('M')

# 儲存樣本外預測結果
oos_predictions = {target: [] for target in target_columns}

# 每月處理數據
for month, monthly_data in step3_df.groupby('month'):
    print(f"Processing month: {month}")

    # 提取特徵矩陣 X 和多目標變數 y
    X = monthly_data[feature_columns].to_numpy()
    y_all = monthly_data[target_columns].to_numpy()

    # 將資料分為 60% 訓練集、20% 驗證集、20% 測試集
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_all, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # 隨機分配到 5 個區塊
    blocks = np.array_split(np.random.permutation(len(X_train)), 5)

    # 對於每個目標變數，進行五折交叉驗證
    for target_idx, target in enumerate(target_columns):
        print(f"Training models for target: {target}")

        # 訓練資料集
        train_data = lgb.Dataset(X_train, label=y_train[:, target_idx])

        # 使用固定迭代次數訓練模型
        model = lgb.train(
            params,
            train_data,
            num_boost_round=TREES
        )

        # 使用固定迭代次數進行預測
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        # 儲存測試結果
        oos_predictions[target].append({
            'month': month,
            'y_val_pred': y_val_pred,
            'y_test_pred': y_test_pred,
            'y_test_actual': y_test[:, target_idx]
        })

# 最終的樣本外預測結果儲存到 DataFrame 中便於分析
oos_df = pd.DataFrame({
    'month': [entry['month'] for target in target_columns for entry in oos_predictions[target]],
    'target': [target for target in target_columns for _ in oos_predictions[target]],
    'y_val_pred': [entry['y_val_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_pred': [entry['y_test_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_actual': [entry['y_test_actual'] for target in target_columns for entry in oos_predictions[target]]
})

# 查看結果
print(oos_df.head())

------

In [ ]:
# 生成葉節點成員矩陣
def getD(tree):
    '''
    用來生成葉節點成員矩陣 D
    如果樣本 f 和 c 位於同一葉節點，則 D[f, c] = 1；否則為 0
    '''
    tree = np.array(tree)  # 確保 tree 是 NumPy 陣列
    D = (tree[:, None] == tree[None, :]).astype(int)
    return D

##### 根據 Geertsema & Lu (2022) 的公式進行計算，計算過程涉及多棵樹的預測和觀測之間的權重更新，目的是模擬一種相對估值模型。

In [ ]:
# ones：建立全為 1 的矩陣，方便矩陣操作。
# I：單位矩陣，用於保持數值計算的穩定性。
# P, p, diff, G, g, D, W：儲存每棵樹中各步驟的計算結果。
N = OBSERVATIONS  # 樣本數量
M = TREES         # 樹的數量
_lambda = LEARNING_RATE  # 學習率

ones = np.ones((N, N))  # 全部為 1 的矩陣
I = np.identity(N)      # 單位矩陣 (N x N)

# 初始化各變數的列表
P, p, diff, G, g, D, W = [[None] * M for _ in range(7)]

In [ ]:
# 迭代第一棵樹的初始化 (i=0)
P[0] = (1 / N) * ones  # 初始化 P[0] 矩陣 (方程 11)
v = y.reshape(N, 1).copy()  # 目標值 y 的向量化

p[0] = v.T @ P[0]  # 初始條件下的 p[0] 計算 (方程 12)
t = np.reshape(tree[0], (1, N))
diff[0] = t - p[0]  # 確認 p[0] 與 tree[0] 的一致性
assert np.allclose(np.mean(tree[0]), np.mean(p[0]), atol=1e-6) # 驗證差異接近零

G[0] = P[0]  # 初始 G[0] 設為 P[0] (方程 13)
g[0] = v.T @ G[0]  # 初始 g[0] 計算 (方程 14)
D[0] = np.ones((N, N))  # 第一棵樹的 D 矩陣，所有樣本在同一葉節點
# P[0]：初始條件下的權重矩陣。
# p[0]：根據初始條件的預測值。
# diff[0]：檢查 p[0] 是否與第一棵樹的預測一致。
# D[0]：所有觀測分配到同一葉節點。

In [ ]:
# 迭代樹 (i ≥ 1)
for i in range(1, M):
    D[i] = getD(tree[i])  # 使用 getD 函數生成 D[i] 矩陣
    W[i] = D[i] / (ones @ D[i])  # W[i] 矩陣計算 (方程 15)

    P[i] = _lambda * (W[i] @ (I - G[i-1]))  # P[i] 計算 (方程 16)
    p[i] = v.T @ P[i].T @ W[i]  # p[i] 計算 (方程 17)

    t = np.reshape(tree[i], (1, N))
    diff[i] = (t * _lambda) - (p[i])  # 檢查差異
    assert(np.allclose(diff[i], np.zeros((1, N)), atol=1e-5))  # 確認差異接近零

    G[i] = G[i-1] + P[i]  # G[i] 累積計算 (方程 18)
    g[i] = v.T @ G[i].T  # g[i] 計算 (方程 19)
# D[i]：第 i 棵樹的葉節點成員矩陣，使用 getD 函數生成。
# W[i]：權重矩陣（方程 15）。
# P[i]：當前迭代的權重更新（方程 16）。
# p[i]：第 i 棵樹的預測值（方程 17）。
# diff[i]：檢查 p[i] 是否與樹的預測一致。
# G[i] 和 g[i]：累積矩陣和對應的向量更新（方程 18 和 19）。

##### 通過計算權重來實現類似 GBM 的預測，並將預測結果與實際的 GBM 預測進行對比，以檢查差異

In [ ]:
# 初始化 L 矩陣
# L 矩陣儲存每棵樹的葉節點分配信息
L = D.copy()

# 在第一棵樹中，所有觀測都分配到同一葉節點
L[0] = np.ones((N, N))  # 第一棵樹的所有值均為 1

In [ ]:
# 利用計算出的權重進行 GBM 預測
K = np.zeros((N, N))
for i in range(0, M):
    # 方程 20：累加每棵樹的預測權重
    K = K + P[i].T @ (L[i] / (ones @ L[i]))

In [ ]:
# 方程 21：計算最終預測值向量 k
k = v.T @ K

# 檢查 k 與 y_hat 的差異
diff_pred = k - y_hat
print("===CHECK===: difference in GBM predicted ", np.round(diff_pred,6))

# 檢查差異是否在允許範圍內
assert(np.allclose(diff_pred, np.zeros((1, N)), atol=1e-5))
print("checks complete")

In [ ]:
-